In [1]:
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", None)

In [3]:
enrol_df = pd.read_csv("enrolment.csv")
demo_df = pd.read_csv("demographic.csv")
bio_df = pd.read_csv("biometric.csv")

In [4]:
all_dfs = [enrol_df, demo_df, bio_df]

In [5]:
enrol_df.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
0,01-09-2025,Gujarat,Ahmadabad,380054,1,0,0
1,01-09-2025,Gujarat,Ahmadabad,382350,3,0,0
2,01-09-2025,Gujarat,Ahmedabad,380005,4,0,0
3,01-09-2025,Gujarat,Ahmedabad,380008,2,1,0
4,01-09-2025,Gujarat,Ahmedabad,380022,1,3,0


In [6]:
enrol_df["date"] = pd.to_datetime(enrol_df["date"], dayfirst=True)
demo_df["date"] = pd.to_datetime(demo_df["date"], dayfirst=True)
bio_df["date"] = pd.to_datetime(bio_df["date"], dayfirst=True)

In [7]:
print("Initial district counts:")
print(enrol_df["district"].nunique(), demo_df["district"].nunique(), bio_df["district"].nunique())

Initial district counts:
40 39 39


In [8]:
enrol_df["district"].value_counts().sort_index()

district
Ahmadabad           897
Ahmedabad          4812
Amreli             1477
Anand              2362
Arvalli             815
Banas Kantha        194
Banaskantha        1921
Bharuch            1657
Bhavnagar          2255
Botad               508
Chhotaudepur        662
Dahod              1068
Dang                  4
Devbhumi Dwarka     695
Dohad               641
Gandhinagar        1478
Gir Somnath         920
Jamnagar           2013
Junagadh           2219
Kachchh            2143
Kheda              2409
Mahesana           1969
Mahisagar           752
Morbi               597
Narmada             585
Navsari            1159
Panch Mahals         83
Panchmahals        1435
Patan              1169
Porbandar           636
Rajkot             2483
Sabar Kantha        166
Sabarkantha        2046
Surat              3546
Surendra Nagar     1799
Surendranagar       140
Tapi                915
The Dangs           269
Vadodara           3804
Valsad             1528
Name: count, dtype: int64

In [9]:
bio_df["district"].value_counts().sort_index()

district
Ahmadabad           8489
Ahmedabad          17971
Amreli              7386
Anand              12962
Arvalli             4011
Banas Kantha         895
Banaskantha         5882
Bharuch             8514
Bhavnagar           9215
Botad               2774
Chhotaudepur        3087
Dahod               3378
Devbhumi Dwarka     3300
Dohad               2730
Gandhinagar         8424
Gir Somnath         3561
Jamnagar            9762
Junagadh           10626
Kachchh            10911
Kheda              10748
Mahesana           10002
Mahisagar           3337
Morbi               2225
Narmada             2681
Navsari             7866
Panch Mahals         885
Panchmahals         6123
Patan               4596
Porbandar           2987
Rajkot             10691
Sabar Kantha        1417
Sabarkantha         9209
Surat              16564
Surendra Nagar      6890
Surendranagar       1577
Tapi                4404
The Dangs           1025
Vadodara           16301
Valsad              7194
Name: count, dty

In [10]:
demo_df["district"].value_counts().sort_index()

district
Ahmadabad          3167
Ahmedabad          7842
Amreli             3094
Anand              5247
Arvalli            1650
Banas Kantha        276
Banaskantha        2651
Bharuch            3647
Bhavnagar          4006
Botad              1048
Chhotaudepur       1324
Dahod              1473
Devbhumi Dwarka    1368
Dohad               994
Gandhinagar        3660
Gir Somnath        1432
Jamnagar           4240
Junagadh           4477
Kachchh            4700
Kheda              4620
Mahesana           4168
Mahisagar          1300
Morbi               945
Narmada            1132
Navsari            3316
Panch Mahals        294
Panchmahals        2635
Patan              1944
Porbandar          1296
Rajkot             4611
Sabar Kantha        483
Sabarkantha        4055
Surat              7083
Surendra Nagar     3009
Surendranagar       574
Tapi               1784
The Dangs           423
Vadodara           7010
Valsad             3113
Name: count, dtype: int64

In [11]:
import re

def clean_district_name(name):
    """Basic text cleaning: lowercase, strip, remove *, normalize spaces"""
    if pd.isna(name):
        return name
    name = str(name).lower().strip()
    name = name.replace('*', '') 
    name = re.sub(r'\s+', ' ', name) 
    name = name.strip()
    return name

print("=== Step 1: Basic Text Cleaning ===")
for df_name, df in [("Enrolment", enrol_df), ("Demographic", demo_df), ("Biometric", bio_df)]:
    before = df['district'].nunique()
    df['district'] = df['district'].apply(clean_district_name)
    after = df['district'].nunique()
    print(f"  {df_name}: {before} -> {after} unique districts")

=== Step 1: Basic Text Cleaning ===
  Enrolment: 40 -> 40 unique districts
  Demographic: 39 -> 39 unique districts
  Biometric: 39 -> 39 unique districts


In [39]:
cleanup_map = {
    "ahmadabad": "ahmedabad",
    "ahmed abad": "ahmedabad",
    "banaskantha": "banas kantha",
    "sabarkantha": "sabar kantha", 
    "panchmahal": "panch mahals",
    "panchmahals": "panch mahals",
    "dahod": "dohad",
    "mahesana": "mahesana",
    "mehsana": "mahesana",
    "surendra nagar": "surendranagar",
}

print("\n=== Step 2: Canonical Mapping ===")
for df_name, df in [("Enrolment", enrol_df), ("Demographic", demo_df), ("Biometric", bio_df)]:
    replacements = {}
    for old_name, new_name in cleanup_map.items():
        if old_name == new_name:
            continue
        count = (df['district'] == old_name).sum()
        if count > 0:
            replacements[old_name] = (new_name, count)
    
    df['district'] = df['district'].replace(cleanup_map)
    
    if replacements:
        print(f"\n{df_name}:")
        for old, (new, cnt) in sorted(replacements.items()):
            print(f"  '{old}' -> '{new}' ({cnt:,} rows)")
    else:
        print(f"\n{df_name}: No additional replacements needed")

print("\n=== Step 3: Final District Counts ===")
print(enrol_df["district"].nunique(), demo_df["district"].nunique(), bio_df["district"].nunique())


=== Step 2: Canonical Mapping ===

Enrolment: No additional replacements needed

Demographic: No additional replacements needed

Biometric: No additional replacements needed

=== Step 3: Final District Counts ===
34 33 33


In [40]:
all_districts = sorted(set(enrol_df['district'].unique()) | 
                       set(demo_df['district'].unique()) | 
                       set(bio_df['district'].unique()))
print(f"Total unique districts across all datasets: {len(all_districts)}")
for d in all_districts:
    print(f"  - {d}")

Total unique districts across all datasets: 34
  - ahmedabad
  - amreli
  - anand
  - arvalli
  - banas kantha
  - bharuch
  - bhavnagar
  - botad
  - chhotaudepur
  - dang
  - devbhumi dwarka
  - dohad
  - gandhinagar
  - gir somnath
  - jamnagar
  - junagadh
  - kachchh
  - kheda
  - mahesana
  - mahisagar
  - morbi
  - narmada
  - navsari
  - panch mahals
  - patan
  - porbandar
  - rajkot
  - sabar kantha
  - surat
  - surendranagar
  - tapi
  - the dangs
  - vadodara
  - valsad


In [41]:
print(enrol_df["pincode"].nunique(), demo_df["pincode"].nunique(), bio_df["pincode"].nunique())

1023 1035 1038


In [42]:
enrol_pincodes = set(enrol_df["pincode"].dropna())
demo_pincodes = set(demo_df["pincode"].dropna())
bio_pincodes = set(bio_df["pincode"].dropna())

print("Enrollment pincodes:", len(enrol_pincodes))
print("Demographic pincodes:", len(demo_pincodes))
print("Biometric pincodes:", len(bio_pincodes))
print()

only_in_enrol = enrol_pincodes - demo_pincodes - bio_pincodes
only_in_demo = demo_pincodes - enrol_pincodes - bio_pincodes
only_in_bio = bio_pincodes - enrol_pincodes - demo_pincodes
in_all = enrol_pincodes & demo_pincodes & bio_pincodes

print(f"Pincodes only in enrollment: {len(only_in_enrol)}")
print(f"Pincodes only in demographic: {len(only_in_demo)}")
print(f"Pincodes only in biometric: {len(only_in_bio)}")
print(f"Pincodes in all three: {len(in_all)}")
print()

print("NULL pincodes:")
print(f"Enrollment: {enrol_df['pincode'].isna().sum()}")
print(f"Demographic: {demo_df['pincode'].isna().sum()}")
print(f"Biometric: {bio_df['pincode'].isna().sum()}")

Enrollment pincodes: 1023
Demographic pincodes: 1035
Biometric pincodes: 1038

Pincodes only in enrollment: 0
Pincodes only in demographic: 0
Pincodes only in biometric: 3
Pincodes in all three: 1023

NULL pincodes:
Enrollment: 0
Demographic: 0
Biometric: 0


In [43]:
for df in all_dfs:
    df["month"] = df["date"].dt.month

In [44]:
enrol_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56231 entries, 0 to 56230
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            56231 non-null  datetime64[ns]
 1   state           56231 non-null  object        
 2   district        56231 non-null  object        
 3   pincode         56231 non-null  int64         
 4   age_0_5         56231 non-null  int64         
 5   age_5_17        56231 non-null  int64         
 6   age_18_greater  56231 non-null  int64         
 7   month           56231 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int64(4), object(2)
memory usage: 3.2+ MB


In [45]:
demo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110091 entries, 0 to 110090
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   date           110091 non-null  datetime64[ns]
 1   state          110091 non-null  object        
 2   district       110091 non-null  object        
 3   pincode        110091 non-null  int64         
 4   demo_age_5_17  110091 non-null  int64         
 5   demo_age_17_   110091 non-null  int64         
 6   month          110091 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int64(3), object(2)
memory usage: 5.5+ MB


In [46]:
bio_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260600 entries, 0 to 260599
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   date          260600 non-null  datetime64[ns]
 1   state         260600 non-null  object        
 2   district      260600 non-null  object        
 3   pincode       260600 non-null  int64         
 4   bio_age_5_17  260600 non-null  int64         
 5   bio_age_17_   260600 non-null  int64         
 6   month         260600 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int64(3), object(2)
memory usage: 12.9+ MB


In [47]:
enrol_df.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater,month
0,2025-09-01,Gujarat,ahmedabad,380054,1,0,0,9
1,2025-09-01,Gujarat,ahmedabad,382350,3,0,0,9
2,2025-09-01,Gujarat,ahmedabad,380005,4,0,0,9
3,2025-09-01,Gujarat,ahmedabad,380008,2,1,0,9
4,2025-09-01,Gujarat,ahmedabad,380022,1,3,0,9


In [48]:
demo_df.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_,month
0,2025-03-01,Gujarat,gandhinagar,382835,18,85,3
1,2025-03-01,Gujarat,rajkot,360030,13,159,3
2,2025-03-01,Gujarat,kachchh,370615,33,193,3
3,2025-03-01,Gujarat,rajkot,360330,13,192,3
4,2025-03-01,Gujarat,kachchh,370490,44,174,3


In [49]:
bio_df.head()

,date,state,district,pincode,bio_age_5_17,bio_age_17_,month
0,2025-12-29,Gujarat,ahmedabad,380015,1,1,12
1,2025-12-29,Gujarat,ahmedabad,380018,0,1,12
2,2025-12-29,Gujarat,ahmedabad,380019,0,1,12
3,2025-12-29,Gujarat,ahmedabad,380024,0,1,12
4,2025-12-29,Gujarat,ahmedabad,380060,0,1,12


In [50]:
enrol_agg = enrol_df.groupby(["state", "district", "month"])[["age_0_5", "age_5_17", "age_18_greater"]].sum().reset_index()
demo_agg = demo_df.groupby(["state", "district", "month"])[["demo_age_5_17", "demo_age_17_"]].sum().reset_index()
bio_agg = bio_df.groupby(["state", "district", "month"])[["bio_age_5_17", "bio_age_17_"]].sum().reset_index()

combined_df = enrol_agg.merge(demo_agg, on = ["state", "district", "month"], how = "left").merge(bio_agg, on = ["state", "district", "month"], how = "left")
combined_df.fillna(0, inplace=True)
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_
0,Gujarat,ahmedabad,1,774,348,73,365.0,4226.0,1775.0,2270.0
1,Gujarat,ahmedabad,4,77,26,19,1594.0,13660.0,20472.0,32913.0
2,Gujarat,ahmedabad,5,756,347,97,1943.0,14425.0,39999.0,65694.0
3,Gujarat,ahmedabad,6,2432,2280,165,1168.0,8168.0,21259.0,43037.0
4,Gujarat,ahmedabad,7,3111,1979,216,1360.0,10424.0,22353.0,44821.0


In [51]:
combined_df["E"] = combined_df["age_0_5"] + combined_df["age_5_17"] + combined_df["age_18_greater"]
combined_df["DU"] = combined_df["demo_age_5_17"] + combined_df["demo_age_17_"]
combined_df["BU"] = combined_df["bio_age_5_17"] + combined_df["bio_age_17_"]
combined_df["U"] = combined_df["DU"] + combined_df["BU"]
combined_df["T"] = combined_df["E"] + combined_df["U"]
combined_df.head(20)

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T
0,Gujarat,ahmedabad,1,774,348,73,365.0,4226.0,1775.0,2270.0,1195,4591.0,4045.0,8636.0,9831.0
1,Gujarat,ahmedabad,4,77,26,19,1594.0,13660.0,20472.0,32913.0,122,15254.0,53385.0,68639.0,68761.0
2,Gujarat,ahmedabad,5,756,347,97,1943.0,14425.0,39999.0,65694.0,1200,16368.0,105693.0,122061.0,123261.0
3,Gujarat,ahmedabad,6,2432,2280,165,1168.0,8168.0,21259.0,43037.0,4877,9336.0,64296.0,73632.0,78509.0
4,Gujarat,ahmedabad,7,3111,1979,216,1360.0,10424.0,22353.0,44821.0,5306,11784.0,67174.0,78958.0,84264.0
5,Gujarat,ahmedabad,9,4177,2148,72,2714.0,24795.0,19732.0,40159.0,6397,27509.0,59891.0,87400.0,93797.0
6,Gujarat,ahmedabad,10,1668,764,72,1968.0,13806.0,6759.0,16220.0,2504,15774.0,22979.0,38753.0,41257.0
7,Gujarat,ahmedabad,11,3917,1533,168,4293.0,43039.0,19602.0,34888.0,5618,47332.0,54490.0,101822.0,107440.0
8,Gujarat,ahmedabad,12,2516,677,127,5048.0,56340.0,21688.0,30676.0,3320,61388.0,52364.0,113752.0,117072.0
9,Gujarat,amreli,1,280,30,9,43.0,738.0,238.0,284.0,319,781.0,522.0,1303.0,1622.0


In [52]:
district_monthly_counts = combined_df.groupby(["district", "month"]).agg(total_months = ("month", "count"), active_months = ("T", lambda x : (x > 0).sum())).reset_index()
district_monthly_counts.head()

,district,month,total_months,active_months
0,ahmedabad,1,1,1
1,ahmedabad,4,1,1
2,ahmedabad,5,1,1
3,ahmedabad,6,1,1
4,ahmedabad,7,1,1


In [53]:
district_monthly_counts["zero_months"] = district_monthly_counts["total_months"] - district_monthly_counts["active_months"]
district_monthly_counts["activity_ratio"] = district_monthly_counts["active_months"] / district_monthly_counts["total_months"]
district_monthly_counts["zero_month_ratio"] = district_monthly_counts["zero_months"] / district_monthly_counts["total_months"]

combined_df = combined_df.merge(district_monthly_counts[["district", "month", "activity_ratio", "zero_month_ratio"]], on = ["district", "month"], how = "left")
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio
0,Gujarat,ahmedabad,1,774,348,73,365.0,4226.0,1775.0,2270.0,1195,4591.0,4045.0,8636.0,9831.0,1.0,0.0
1,Gujarat,ahmedabad,4,77,26,19,1594.0,13660.0,20472.0,32913.0,122,15254.0,53385.0,68639.0,68761.0,1.0,0.0
2,Gujarat,ahmedabad,5,756,347,97,1943.0,14425.0,39999.0,65694.0,1200,16368.0,105693.0,122061.0,123261.0,1.0,0.0
3,Gujarat,ahmedabad,6,2432,2280,165,1168.0,8168.0,21259.0,43037.0,4877,9336.0,64296.0,73632.0,78509.0,1.0,0.0
4,Gujarat,ahmedabad,7,3111,1979,216,1360.0,10424.0,22353.0,44821.0,5306,11784.0,67174.0,78958.0,84264.0,1.0,0.0


In [54]:
print("Columns:", combined_df.columns.tolist())

Columns: ['state', 'district', 'month', 'age_0_5', 'age_5_17', 'age_18_greater', 'demo_age_5_17', 'demo_age_17_', 'bio_age_5_17', 'bio_age_17_', 'E', 'DU', 'BU', 'U', 'T', 'activity_ratio', 'zero_month_ratio']


In [55]:
district_volume_metrics = combined_df.groupby(["state", "district"]).agg(avg_monthly_enrolment = ("E", "mean"),
                                               monthly_valatility = ("T", lambda x: x.std(ddof=0) / x.mean() if x.mean() > 0 else 0),
                                               peak_load_ratio = ("T", lambda x: x.max() / x.mean() if x.mean() > 0 else 0)).reset_index()


combined_df = combined_df.merge(district_volume_metrics, on=["state", "district"], how="left")
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio,avg_monthly_enrolment,monthly_valatility,peak_load_ratio
0,Gujarat,ahmedabad,1,774,348,73,365.0,4226.0,1775.0,2270.0,1195,4591.0,4045.0,8636.0,9831.0,1.0,0.0,3393.222222,0.429681,1.531844
1,Gujarat,ahmedabad,4,77,26,19,1594.0,13660.0,20472.0,32913.0,122,15254.0,53385.0,68639.0,68761.0,1.0,0.0,3393.222222,0.429681,1.531844
2,Gujarat,ahmedabad,5,756,347,97,1943.0,14425.0,39999.0,65694.0,1200,16368.0,105693.0,122061.0,123261.0,1.0,0.0,3393.222222,0.429681,1.531844
3,Gujarat,ahmedabad,6,2432,2280,165,1168.0,8168.0,21259.0,43037.0,4877,9336.0,64296.0,73632.0,78509.0,1.0,0.0,3393.222222,0.429681,1.531844
4,Gujarat,ahmedabad,7,3111,1979,216,1360.0,10424.0,22353.0,44821.0,5306,11784.0,67174.0,78958.0,84264.0,1.0,0.0,3393.222222,0.429681,1.531844


In [56]:
district_update_burden = combined_df.groupby(["state", "district"]).agg(avg_monthly_enrollments = ("E", "sum"), avg_monthly_demo_updates = ("DU", "sum"), avg_monthly_bio_updates = ("BU", "sum")).reset_index()

district_update_burden["U"] = district_update_burden["avg_monthly_demo_updates"] + district_update_burden["avg_monthly_bio_updates"]
district_update_burden["biometric_burden"] = district_update_burden["avg_monthly_bio_updates"] / (district_update_burden["avg_monthly_bio_updates"] + district_update_burden["avg_monthly_demo_updates"])
district_update_burden["update_dominant"] = np.where(district_update_burden["U"] > district_update_burden["avg_monthly_enrollments"], 1, 0)
district_update_burden["enrollment_update_balance"] = district_update_burden["avg_monthly_enrollments"] / (district_update_burden["avg_monthly_enrollments"] + district_update_burden["U"])

combined_df = combined_df.merge(district_update_burden[["state", "district", "biometric_burden", "update_dominant", "enrollment_update_balance"]], on=["state", "district"], how="left")
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio,avg_monthly_enrolment,monthly_valatility,peak_load_ratio,biometric_burden,update_dominant,enrollment_update_balance
0,Gujarat,ahmedabad,1,774,348,73,365.0,4226.0,1775.0,2270.0,1195,4591.0,4045.0,8636.0,9831.0,1.0,0.0,3393.222222,0.429681,1.531844,0.698212,1,0.04217
1,Gujarat,ahmedabad,4,77,26,19,1594.0,13660.0,20472.0,32913.0,122,15254.0,53385.0,68639.0,68761.0,1.0,0.0,3393.222222,0.429681,1.531844,0.698212,1,0.04217
2,Gujarat,ahmedabad,5,756,347,97,1943.0,14425.0,39999.0,65694.0,1200,16368.0,105693.0,122061.0,123261.0,1.0,0.0,3393.222222,0.429681,1.531844,0.698212,1,0.04217
3,Gujarat,ahmedabad,6,2432,2280,165,1168.0,8168.0,21259.0,43037.0,4877,9336.0,64296.0,73632.0,78509.0,1.0,0.0,3393.222222,0.429681,1.531844,0.698212,1,0.04217
4,Gujarat,ahmedabad,7,3111,1979,216,1360.0,10424.0,22353.0,44821.0,5306,11784.0,67174.0,78958.0,84264.0,1.0,0.0,3393.222222,0.429681,1.531844,0.698212,1,0.04217


In [57]:
cols_to_drop = [c for c in combined_df.columns if c.endswith('_y')]
if cols_to_drop:
    combined_df.drop(cols_to_drop, axis=1, inplace=True)
    combined_df.rename(columns={c: c.replace('_x', '') for c in combined_df.columns if c.endswith('_x')}, inplace=True)
print("Columns after cleanup:", combined_df.columns.tolist())

Columns after cleanup: ['state', 'district', 'month', 'age_0_5', 'age_5_17', 'age_18_greater', 'demo_age_5_17', 'demo_age_17_', 'bio_age_5_17', 'bio_age_17_', 'E', 'DU', 'BU', 'U', 'T', 'activity_ratio', 'zero_month_ratio', 'avg_monthly_enrolment', 'monthly_valatility', 'peak_load_ratio', 'biometric_burden', 'update_dominant', 'enrollment_update_balance']


In [58]:
combined_df = combined_df.groupby(["state", "district"], as_index = False).first()
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio,avg_monthly_enrolment,monthly_valatility,peak_load_ratio,biometric_burden,update_dominant,enrollment_update_balance
0,Gujarat,ahmedabad,1,774,348,73,365.0,4226.0,1775.0,2270.0,1195,4591.0,4045.0,8636.0,9831.0,1.0,0.0,3393.222222,0.429681,1.531844,0.698212,1,0.042170
1,Gujarat,amreli,1,280,30,9,43.0,738.0,238.0,284.0,319,781.0,522.0,1303.0,1622.0,1.0,0.0,654.625000,0.428469,1.498423,0.766217,1,0.056199
2,Gujarat,anand,1,247,39,27,60.0,848.0,472.0,559.0,313,908.0,1031.0,1939.0,2252.0,1.0,0.0,880.888889,0.437137,1.448305,0.833725,1,0.043913
3,Gujarat,arvalli,1,111,15,0,37.0,181.0,270.0,160.0,126,218.0,430.0,648.0,774.0,1.0,0.0,348.625000,0.500847,1.521213,0.821669,1,0.067887
4,Gujarat,banas kantha,1,449,226,9,100.0,1791.0,1214.0,1271.0,684,1891.0,2485.0,4376.0,5060.0,1.0,0.0,3466.700000,0.456390,1.672763,0.695615,1,0.107562


In [59]:
def normalize(x):
    maxx, minx = x.max(), x.min()
    if maxx == minx:
        return x * 0 + 0.5
    normalized = (x - minx) / (maxx - minx)
    return normalized

def inverse_normalize(x):
    inversed = 1 - normalize(x)
    return inversed

In [60]:
combined_df["access"] = (combined_df["activity_ratio"] + normalize(combined_df["avg_monthly_enrolment"])) / 2
combined_df["responsiveness"] = normalize(combined_df["U"] / (combined_df["E"] + combined_df["U"]))
combined_df["inclusion"] = normalize((combined_df["age_0_5"] + combined_df["age_5_17"]) / combined_df["E"])
combined_df["stability"] = (inverse_normalize(combined_df["monthly_valatility"]) + inverse_normalize(combined_df["peak_load_ratio"])) / 2
combined_df["visibility"] = combined_df["activity_ratio"]

combined_df["DEI"] = (combined_df["access"] + combined_df["responsiveness"] + combined_df["inclusion"] + combined_df["stability"] + combined_df["visibility"]) / 5
combined_df["ASS"] = (inverse_normalize(combined_df["activity_ratio"]) + inverse_normalize(combined_df["avg_monthly_enrolment"])) / 2
combined_df["UBS"] = (normalize(combined_df["biometric_burden"]) + normalize(combined_df["update_dominant"])) / 2
combined_df["SRS"] = (normalize(combined_df["monthly_valatility"]) + normalize(combined_df["zero_month_ratio"])) / 2

combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio,avg_monthly_enrolment,monthly_valatility,peak_load_ratio,biometric_burden,update_dominant,enrollment_update_balance,access,responsiveness,inclusion,stability,visibility,DEI,ASS,UBS,SRS
0,Gujarat,ahmedabad,1,774,348,73,365.0,4226.0,1775.0,2270.0,1195,4591.0,4045.0,8636.0,9831.0,1.0,0.0,3393.222222,0.429681,1.531844,0.698212,1,0.042170,0.988786,0.959533,0.821623,0.477586,1.0,0.849506,0.261214,0.771599,0.574110
1,Gujarat,amreli,1,280,30,9,43.0,738.0,238.0,284.0,319,781.0,522.0,1303.0,1622.0,1.0,0.0,654.625000,0.428469,1.498423,0.766217,1,0.056199,0.570816,0.877483,0.917618,0.497145,1.0,0.772612,0.679184,0.869715,0.572624
2,Gujarat,anand,1,247,39,27,60.0,848.0,472.0,559.0,313,908.0,1031.0,1939.0,2252.0,1.0,0.0,880.888889,0.437137,1.448305,0.833725,1,0.043913,0.605349,0.940491,0.748115,0.513626,1.0,0.761516,0.644651,0.967114,0.583245
3,Gujarat,arvalli,1,111,15,0,37.0,181.0,270.0,160.0,126,218.0,430.0,648.0,774.0,1.0,0.0,348.625000,0.500847,1.521213,0.821669,1,0.067887,0.524114,0.914490,1.000000,0.396144,1.0,0.766950,0.725886,0.949720,0.661300
4,Gujarat,banas kantha,1,449,226,9,100.0,1791.0,1214.0,1271.0,684,1891.0,2485.0,4376.0,5060.0,1.0,0.0,3466.700000,0.456390,1.672763,0.695615,1,0.107562,1.000000,0.944652,0.961579,0.368658,1.0,0.854978,0.250000,0.767852,0.606833


In [61]:
combined_df.drop(["access", "responsiveness", "inclusion", "stability", "visibility"], axis=1, inplace=True)
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio,avg_monthly_enrolment,monthly_valatility,peak_load_ratio,biometric_burden,update_dominant,enrollment_update_balance,DEI,ASS,UBS,SRS
0,Gujarat,ahmedabad,1,774,348,73,365.0,4226.0,1775.0,2270.0,1195,4591.0,4045.0,8636.0,9831.0,1.0,0.0,3393.222222,0.429681,1.531844,0.698212,1,0.042170,0.849506,0.261214,0.771599,0.574110
1,Gujarat,amreli,1,280,30,9,43.0,738.0,238.0,284.0,319,781.0,522.0,1303.0,1622.0,1.0,0.0,654.625000,0.428469,1.498423,0.766217,1,0.056199,0.772612,0.679184,0.869715,0.572624
2,Gujarat,anand,1,247,39,27,60.0,848.0,472.0,559.0,313,908.0,1031.0,1939.0,2252.0,1.0,0.0,880.888889,0.437137,1.448305,0.833725,1,0.043913,0.761516,0.644651,0.967114,0.583245
3,Gujarat,arvalli,1,111,15,0,37.0,181.0,270.0,160.0,126,218.0,430.0,648.0,774.0,1.0,0.0,348.625000,0.500847,1.521213,0.821669,1,0.067887,0.766950,0.725886,0.949720,0.661300
4,Gujarat,banas kantha,1,449,226,9,100.0,1791.0,1214.0,1271.0,684,1891.0,2485.0,4376.0,5060.0,1.0,0.0,3466.700000,0.456390,1.672763,0.695615,1,0.107562,0.854978,0.250000,0.767852,0.606833


In [62]:
print(f"Total districts: {len(combined_df)}")
print(f"\nScore summary statistics:")
for col in ['DEI', 'ASS', 'UBS', 'SRS']:
    print(f"  {col}: min={combined_df[col].min():.4f}, max={combined_df[col].max():.4f}, mean={combined_df[col].mean():.4f}")

for col in ['DEI', 'ASS', 'UBS', 'SRS']:
    assert combined_df[col].min() >= 0, f"{col} has negative values"
    assert combined_df[col].max() <= 1, f"{col} exceeds 1"
print("\n✓ All scores in valid [0, 1] range")

Total districts: 34

Score summary statistics:
  DEI: min=0.5679, max=0.8654, mean=0.7667
  ASS: min=0.2500, max=0.7500, mean=0.6175
  UBS: min=0.5000, max=1.0000, mean=0.8440
  SRS: min=0.2500, max=0.7500, mean=0.5864

✓ All scores in valid [0, 1] range


In [63]:
combined_df.to_csv("gujarat_district_analysis.csv", index=False)
final_df = combined_df[["state", "district", "DEI", "ASS", "UBS", "SRS"]]
final_df.to_csv("gujarat_district_final_scores.csv", index=False)
print("✓ Saved gujarat_district_analysis.csv")
print("✓ Saved gujarat_district_final_scores.csv")

✓ Saved gujarat_district_analysis.csv
✓ Saved gujarat_district_final_scores.csv


In [64]:
final_df.sort_values('DEI', ascending=False)

,state,district,DEI,ASS,UBS,SRS
28,Gujarat,surat,0.865387,0.282954,0.500000,0.553203
4,Gujarat,banas kantha,0.854978,0.250000,0.767852,0.606833
0,Gujarat,ahmedabad,0.849506,0.261214,0.771599,0.574110
11,Gujarat,dohad,0.846211,0.390122,0.944385,0.551298
26,Gujarat,rajkot,0.838579,0.445785,0.662717,0.550677
16,Gujarat,kachchh,0.820874,0.615315,0.786200,0.530625
6,Gujarat,bhavnagar,0.803757,0.554129,0.845101,0.587707
18,Gujarat,mahesana,0.799875,0.648025,0.844939,0.585759
17,Gujarat,kheda,0.796820,0.628456,0.915715,0.581375
33,Gujarat,valsad,0.790547,0.633764,0.795658,0.553277
